In [1]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score

In [ ]:
client = OpenAI(api_key = 'sk-proj-')

In [3]:
df = pd.read_csv('framingham.csv')

In [4]:
example_row = df.iloc[0]
example_features = example_row.drop(labels='TenYearCHD')
example_label = example_row['TenYearCHD']

In [5]:
test_df = df.iloc[1:21] 

In [6]:
target_sample = test_df['TenYearCHD']
features_sample = test_df.drop(columns=['TenYearCHD'])

In [10]:
def llm_predict(row, example_features, example_label):
    example_input = example_features.to_string(index=True)
    example_output = str(example_label)
    print(f"Example input:\n{example_input}\nExample output:\n{example_output}")

    row_string = row.to_string(index=True)

    prompt = f"""You are a cardiovascular disease risk prediction expert.

Based on structured patient data, predict whether the person is at risk of developing coronary heart disease within 10 years.
Return only 1 (at risk) or 0 (not at risk).

Example:
{example_input}
Answer: {example_output}

Now predict for the following case:
{row_string}
Answer:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a medical expert specialized in cardiovascular risk prediction."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    output = response.choices[0].message.content.strip()
    return int('1' in output)


In [11]:
preds = features_sample.apply(lambda row: llm_predict(row, example_features, example_label), axis=1)

Example input:
male                 1.00
age                 39.00
education            4.00
currentSmoker        0.00
cigsPerDay           0.00
BPMeds               0.00
prevalentStroke      0.00
prevalentHyp         0.00
diabetes             0.00
totChol            195.00
sysBP              106.00
diaBP               70.00
BMI                 26.97
heartRate           80.00
glucose             77.00
Example output:
0.0
Example input:
male                 1.00
age                 39.00
education            4.00
currentSmoker        0.00
cigsPerDay           0.00
BPMeds               0.00
prevalentStroke      0.00
prevalentHyp         0.00
diabetes             0.00
totChol            195.00
sysBP              106.00
diaBP               70.00
BMI                 26.97
heartRate           80.00
glucose             77.00
Example output:
0.0
Example input:
male                 1.00
age                 39.00
education            4.00
currentSmoker        0.00
cigsPerDay           0.00
BPMed

In [12]:
acc = accuracy_score(target_sample, preds)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.20
